In [1]:
import pickle
import random
import numpy
import torch
import os

from Auxiliary_Functions.generate_observation import generate_observations, split_dataset
from Auxiliary_Functions.save_progress import save_progress
from Auxiliary_Functions.matrix_normalization import normalize_max_row_norm

from Denoising_Algorithms.Projectors.DnCNN import DnCNN
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR
from Denoising_Algorithms.DL_Training.loss_functions import IntermediateScaledLoss
from Denoising_Algorithms.DL_Training.training import train_main
from Denoising_Algorithms.Memory_Network.Memory_Net import MemoryNetwork
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR

In [2]:
current_trial = 0

torch.manual_seed(current_trial)
numpy.random.seed(current_trial)
random.seed(current_trial)
torch.cuda.manual_seed(current_trial)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_10.pkl', 'rb') as f:
    gaussian_10 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_20.pkl', 'rb') as f:
    gaussian_20 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_30.pkl', 'rb') as f:
    gaussian_30 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_40.pkl', 'rb') as f:
    gaussian_40 = pickle.load(f)

In [4]:
with open('ILSVRC2012_50x50_small.pkl', 'rb') as f:
    ILSVRC2012 = pickle.load(f)

# We only work with the first 25000 images
img_dataset = ILSVRC2012[:25000]

In [5]:
gaussian_10_normalized = normalize_max_row_norm(gaussian_10)
gaussian_20_normalized = normalize_max_row_norm(gaussian_20)
gaussian_30_normalized = normalize_max_row_norm(gaussian_30)
gaussian_40_normalized = normalize_max_row_norm(gaussian_40)

gaussian_10_observations = generate_observations(img_dataset, gaussian_10_normalized, 0.025)
gaussian_20_observations = generate_observations(img_dataset, gaussian_20_normalized, 0.025)
gaussian_30_observations = generate_observations(img_dataset, gaussian_30_normalized, 0.025)
gaussian_40_observations = generate_observations(img_dataset, gaussian_40_normalized, 0.025)

gaussian_10_train, gaussian_10_test = split_dataset(gaussian_10_observations, train_ratio = 0.9, seed = 0)
gaussian_20_train, gaussian_20_test = split_dataset(gaussian_20_observations, train_ratio = 0.9, seed = 0)
gaussian_30_train, gaussian_30_test = split_dataset(gaussian_30_observations, train_ratio = 0.9, seed = 0)
gaussian_40_train, gaussian_40_test = split_dataset(gaussian_40_observations, train_ratio = 0.9, seed = 0)

In [6]:
projector = DnCNN(depth = 5)

# Equal Weighting
loss_function = IntermediateScaledLoss(omega = 1.0)

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
# 5 Projections
num_projections = 5

# Folder Path for 5 Projections
proj5_path = 'Memory_Residual-Additive-.025/5 Projections'

gauss_10_proj5_weights, gauss_10_proj5_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj5', display=True)

gauss_20_proj5_weights, gauss_20_proj5_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj5', display=True)

gauss_30_proj5_weights, gauss_30_proj5_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj5', display=True)

gauss_40_proj5_weights, gauss_40_proj5_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj5', display=True)

# Store Best Model Weights and History

save_progress(proj5_path, gauss_10_proj5_weights, gauss_10_proj5_hist, '10% Sampling')
save_progress(proj5_path, gauss_20_proj5_weights, gauss_20_proj5_hist, '20% Sampling')
save_progress(proj5_path, gauss_30_proj5_weights, gauss_30_proj5_hist, '30% Sampling')
save_progress(proj5_path, gauss_40_proj5_weights, gauss_40_proj5_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.15800885087251662
Average test loss: 0.01096521337164773
Epoch 2/300
Average training loss: 0.06364845725562837
Average test loss: 0.009768919814378023
Epoch 3/300
Average training loss: 0.05730780975023905
Average test loss: 0.010604006215102144
Epoch 4/300
Average training loss: 0.05392448871003257
Average test loss: 0.009271428898804718
Epoch 5/300
Average training loss: 0.051685002939568625
Average test loss: 0.00830350274592638
Epoch 6/300
Average training loss: 0.04915047177672386
Average test loss: 0.008877529405885272
Epoch 7/300
Average training loss: 0.04773576867580414
Average test loss: 0.008568395098050435
Epoch 8/300
Average training loss: 0.046410978509320154
Average test loss: 0.008301617874039545
Epoch 9/300
Average training loss: 0.04556328484084871
Average test loss: 0.00795124609851175
Epoch 10/300
Average training loss: 0.04468259781267908
Average test loss: 0.008290607819126711
Epoch 11/

In [8]:
gauss_10_proj5_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj5_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj5_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj5_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj5_psnr = average_PSNR(gauss_10_proj5_model, gauss_10_proj5_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj5_psnr = average_PSNR(gauss_20_proj5_model, gauss_20_proj5_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj5_psnr = average_PSNR(gauss_30_proj5_model, gauss_30_proj5_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj5_psnr = average_PSNR(gauss_40_proj5_model, gauss_40_proj5_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj5_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj5_psnr, f)
with open(os.path.join(proj5_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj5_psnr, f)
with open(os.path.join(proj5_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj5_psnr, f)
with open(os.path.join(proj5_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj5_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 22.16
Average PSNR for Projection Layer 1 across 2500 images: 23.69
Average PSNR for Projection Layer 2 across 2500 images: 24.40
Average PSNR for Projection Layer 3 across 2500 images: 25.04
Average PSNR for Projection Layer 4 across 2500 images: 25.38
Average PSNR for Projection Layer 0 across 2500 images: 23.38
Average PSNR for Projection Layer 1 across 2500 images: 25.78
Average PSNR for Projection Layer 2 across 2500 images: 26.92
Average PSNR for Projection Layer 3 across 2500 images: 27.48
Average PSNR for Projection Layer 4 across 2500 images: 27.83
Average PSNR for Projection Layer 0 across 2500 images: 23.97
Average PSNR for Projection Layer 1 across 2500 images: 26.83
Average PSNR for Projection Layer 2 across 2500 images: 28.10
Average PSNR for Projection Layer 3 across 2500 images: 28.87
Average PSNR for Projection Layer 4 across 2500 images: 29.29
Average PSNR for Projection Layer 0 across 2500 images: 24.24
Average 

In [ ]:
# 15 Projections
num_projections = 15

# Folder Path for 15 Projections
proj15_path = 'Memory_Residual-Additive-.025/15 Projections'

gauss_10_proj15_weights, gauss_10_proj15_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj15', display=True)

gauss_20_proj15_weights, gauss_20_proj15_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj15', display=True)

gauss_30_proj15_weights, gauss_30_proj15_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj15', display=True)

gauss_40_proj15_weights, gauss_40_proj15_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj15', display=True)

# Store Best Model Weights and History

save_progress(proj15_path, gauss_10_proj15_weights, gauss_10_proj15_hist, '10% Sampling')
save_progress(proj15_path, gauss_20_proj15_weights, gauss_20_proj15_hist, '20% Sampling')
save_progress(proj15_path, gauss_30_proj15_weights, gauss_30_proj15_hist, '30% Sampling')
save_progress(proj15_path, gauss_40_proj15_weights, gauss_40_proj15_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 1.883851372083028
Average test loss: 0.012949718395868937
Epoch 2/300
Average training loss: 0.6788029842376709
Average test loss: 0.012264204132888052
Epoch 3/300
Average training loss: 0.4657020842499203
Average test loss: 0.008794440970652634
Epoch 4/300
Average training loss: 0.3670125378767649
Average test loss: 0.008455127709855636
Epoch 5/300
Average training loss: 0.3070465210808648
Average test loss: 0.008775156863033772
Epoch 6/300
Average training loss: 0.2672246738539802
Average test loss: 0.007982154718703694
Epoch 7/300
Average training loss: 0.2382504272196028
Average test loss: 0.009370613079104159
Epoch 8/300
Average training loss: 0.2141343701945411
Average test loss: 0.007845525061918629
Epoch 9/300
Average training loss: 0.19977063177691565
Average test loss: 0.007858407395995326
Epoch 10/300
Average training loss: 0.18668859190411038
Average test loss: 0.007233142123040226
Epoch 11/300
Aver

In [10]:
gauss_10_proj15_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj15_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj15_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj15_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj15_psnr = average_PSNR(gauss_10_proj15_model, gauss_10_proj15_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj15_psnr = average_PSNR(gauss_20_proj15_model, gauss_20_proj15_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj15_psnr = average_PSNR(gauss_30_proj15_model, gauss_30_proj15_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj15_psnr = average_PSNR(gauss_40_proj15_model, gauss_40_proj15_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj15_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj15_psnr, f)
with open(os.path.join(proj15_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj15_psnr, f)
with open(os.path.join(proj15_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj15_psnr, f)
with open(os.path.join(proj15_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj15_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 23.83
Average PSNR for Projection Layer 1 across 2500 images: 27.13
Average PSNR for Projection Layer 2 across 2500 images: 28.67
Average PSNR for Projection Layer 3 across 2500 images: 29.31
Average PSNR for Projection Layer 4 across 2500 images: 29.72
Average PSNR for Projection Layer 5 across 2500 images: 29.97
Average PSNR for Projection Layer 6 across 2500 images: 30.10
Average PSNR for Projection Layer 7 across 2500 images: 30.08
Average PSNR for Projection Layer 8 across 2500 images: 30.40
Average PSNR for Projection Layer 9 across 2500 images: 30.59
Average PSNR for Projection Layer 10 across 2500 images: 30.67
Average PSNR for Projection Layer 11 across 2500 images: 30.75
Average PSNR for Projection Layer 12 across 2500 images: 30.80
Average PSNR for Projection Layer 13 across 2500 images: 30.80
Average PSNR for Projection Layer 14 across 2500 images: 30.81


In [11]:
# 30 Projections
num_projections = 30

# Folder Path for 30 Projections
proj30_path = 'Memory_Residual-Additive-.025/30 Projections'

gauss_10_proj30_weights, gauss_10_proj30_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj30', display=True)

gauss_20_proj30_weights, gauss_20_proj30_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj30', display=True)

gauss_30_proj30_weights, gauss_30_proj30_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj30', display=True)

gauss_40_proj30_weights, gauss_40_proj30_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj30', display=True)

# Store Best Model Weights and History

save_progress(proj30_path, gauss_10_proj30_weights, gauss_10_proj30_hist, '10% Sampling')
save_progress(proj30_path, gauss_20_proj30_weights, gauss_20_proj30_hist, '20% Sampling')
save_progress(proj30_path, gauss_30_proj30_weights, gauss_30_proj30_hist, '30% Sampling')
save_progress(proj30_path, gauss_40_proj30_weights, gauss_40_proj30_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 17.035380304972332
Average test loss: 0.013302623805900415
Epoch 2/300
Average training loss: 9.444275538974338
Average test loss: 0.010775287286274964
Epoch 3/300
Average training loss: 7.104689140743679
Average test loss: 0.009619573559198114
Epoch 4/300
Average training loss: 5.781412685818142
Average test loss: 0.01020106866541836
Epoch 5/300
Average training loss: 5.07729153696696
Average test loss: 0.008771260405580203
Epoch 6/300
Average training loss: 4.846248065524631
Average test loss: 0.008358446300857598
Epoch 7/300
Average training loss: 4.311109864552816
Average test loss: 0.016668826003869375
Epoch 8/300
Average training loss: 4.0164016473558215
Average test loss: 0.008982638282908333
Epoch 9/300
Average training loss: 3.5138334291246203
Average test loss: 0.008243970065895055
Epoch 10/300
Average training loss: 3.017673257827759
Average test loss: 0.007665864098403189
Epoch 11/300
Average traini

In [12]:
gauss_10_proj30_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj30_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj30_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj30_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj30_psnr = average_PSNR(gauss_10_proj30_model, gauss_10_proj30_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj30_psnr = average_PSNR(gauss_20_proj30_model, gauss_20_proj30_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj30_psnr = average_PSNR(gauss_30_proj30_model, gauss_30_proj30_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj30_psnr = average_PSNR(gauss_40_proj30_model, gauss_40_proj30_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj30_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj30_psnr, f)
with open(os.path.join(proj30_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj30_psnr, f)
with open(os.path.join(proj30_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj30_psnr, f)
with open(os.path.join(proj30_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj30_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 21.40
Average PSNR for Projection Layer 1 across 2500 images: 23.13
Average PSNR for Projection Layer 2 across 2500 images: 23.75
Average PSNR for Projection Layer 3 across 2500 images: 24.33
Average PSNR for Projection Layer 4 across 2500 images: 24.55
Average PSNR for Projection Layer 5 across 2500 images: 24.81
Average PSNR for Projection Layer 6 across 2500 images: 25.06
Average PSNR for Projection Layer 7 across 2500 images: 24.75
Average PSNR for Projection Layer 8 across 2500 images: 24.61
Average PSNR for Projection Layer 9 across 2500 images: 24.91
Average PSNR for Projection Layer 10 across 2500 images: 25.02
Average PSNR for Projection Layer 11 across 2500 images: 25.23
Average PSNR for Projection Layer 12 across 2500 images: 25.24
Average PSNR for Projection Layer 13 across 2500 images: 25.49
Average PSNR for Projection Layer 14 across 2500 images: 25.57
Average PSNR for Projection Layer 15 across 2500 images: 25.67
Av

In [13]:
import shutil

def zip_folder(source_folder, output_zip_file):
    # Ensure the output does not contain an extension (shutil will append .zip)
    output_zip_file = output_zip_file.rstrip('.zip')

    # Zip the folder
    shutil.make_archive(output_zip_file, 'zip', source_folder)

# Define the folder to zip and the location of the output file
source_folder = 'Memory_Residual-Additive-.025'  # Replace with the path to your folder
output_zip_file = 'Memory_Residual-Additive-.025.zip'  # Replace with the new zip location without .zip

# Call the function to zip the folder
zip_folder(source_folder, output_zip_file)
